In [1]:
## This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/mltiverse-join-comp/sample_submission.csv
/kaggle/input/mltiverse-join-comp/train.csv
/kaggle/input/mltiverse-join-comp/test.csv


importing data

In [2]:
train_data=pd.read_csv('/kaggle/input/mltiverse-join-comp/train.csv')
test_data=pd.read_csv('/kaggle/input/mltiverse-join-comp/test.csv')
sample=pd.read_csv('/kaggle/input/mltiverse-join-comp/sample_submission.csv')

making some comparisions

In [3]:
sample.shape

(16710, 2)

In [4]:
test_data.shape

(16710, 65)

checking if photo id is the same for both

In [5]:
(test_data['Photo'] == sample['Photo']).all()

True

therefore it is the exact same

checking the number of rows and analysing test data

In [6]:
train_data.shape

(87642, 67)

In [7]:
train_data['Photo'].nunique()

74502

In [8]:
 train_data[['Photo', 'year']].drop_duplicates().shape[0]

87642

keeping only one row per id(the latest year in this case)

first and foremost let me check for test data. the shape, unique ids, photo etc etc


In [9]:
test_data.shape[0]==test_data['ID'].nunique()

True

therefore the ids and photos are linearly mapped, so should only keep one id for training data

In [10]:
# Sort the train_data DataFrame by 'ID' and 'year' in descending order
train_data_sorted = train_data.sort_values(by=['ID', 'year'], ascending=[True, False])

# Drop duplicate 'ID' values, keeping only the first occurrence (which corresponds to the latest year)
train_data_1 = train_data_sorted.drop_duplicates(subset='ID', keep='first')

# Reset the index of the new dataset
train_data_1 = train_data_1.reset_index(drop=True)

In [11]:
train_data_1.shape

(33699, 67)

feature engineering some columns

In [12]:
train_data_1['Position'].unique()

array(['<span class="pos pos29">RES', '<span class="pos pos15">LCM',
       '<span class="pos pos18">CAM', '<span class="pos pos28">SUB', nan,
       '<span class="pos pos0">GK', '<span class="pos pos10">CDM',
       '<span class="pos pos5">CB', '<span class="pos pos4">RCB',
       '<span class="pos pos25">ST', '<span class="pos pos6">LCB',
       '<span class="pos pos14">CM', '<span class="pos pos26">LS',
       '<span class="pos pos9">RDM', '<span class="pos pos13">RCM',
       '<span class="pos pos16">LM', '<span class="pos pos19">LAM',
       '<span class="pos pos11">LDM', '<span class="pos pos3">RB',
       '<span class="pos pos12">RM', '<span class="pos pos21">CF',
       '<span class="pos pos24">RS', '<span class="pos pos23">RW',
       '<span class="pos pos7">LB', '<span class="pos pos27">LW',
       '<span class="pos pos8">LWB', '<span class="pos pos2">RWB',
       '<span class="pos pos17">RAM', '<span class="pos pos22">LF',
       '<span class="pos pos20">RF'], dtype=object)

In [13]:
test_data['Position'].dtype

dtype('O')

In [14]:
train_data_1['Position'] = train_data_1['Position'].astype(str)
train_data_1['Position'] = train_data_1['Position'].str[-3:]

In [15]:
train_data_1['Position'].unique()

array(['RES', 'LCM', 'CAM', 'SUB', 'nan', '>GK', 'CDM', '>CB', 'RCB',
       '>ST', 'LCB', '>CM', '>LS', 'RDM', 'RCM', '>LM', 'LAM', 'LDM',
       '>RB', '>RM', '>CF', '>RS', '>RW', '>LB', '>LW', 'LWB', 'RWB',
       'RAM', '>LF', '>RF'], dtype=object)

trying to remove the '>'

In [16]:
# Remove '>' character from the beginning of each value in 'Position' column
train_data_1['Position'] = train_data_1['Position'].str.lstrip('>')

# Remove rows where 'Position' is NaN
train_data_1 = train_data_1.dropna(subset=['Position'])

removing all the nan values

In [17]:
train_data_1 = train_data_1[train_data_1['Position'] != 'nan']

In [18]:
train_data_1['Position'].unique()

array(['RES', 'LCM', 'CAM', 'SUB', 'GK', 'CDM', 'CB', 'RCB', 'ST', 'LCB',
       'CM', 'LS', 'RDM', 'RCM', 'LM', 'LAM', 'LDM', 'RB', 'RM', 'CF',
       'RS', 'RW', 'LB', 'LW', 'LWB', 'RWB', 'RAM', 'LF', 'RF'],
      dtype=object)

doing the exact same thing for the test dataset now

In [19]:
test_data['Position'] = test_data['Position'].astype(str)
test_data['Position'] = test_data['Position'].str[-3:]  # Remove '>' character from the beginning of each value in 'Position' column
test_data['Position'] = test_data['Position'].str.lstrip('>')
test_data = test_data[test_data['Position'] != 'nan']  # Remove rows where 'Position' is 'nan'

engineering height by converting it completely into inches for training and test data

In [20]:
# Convert feet and inches to inches
train_data_1['Height'] = train_data_1['Height'].apply(lambda x: int(x.split("'")[0]) * 12 + int(x.split("'")[1]))
# Convert 'Height' column to integer data type
train_data_1['Height'] = train_data_1['Height'].astype(float)
# Convert centimeters to inches
test_data['Height'] = test_data['Height'].apply(lambda x: int(x[:-2]) / 2.54)
# Convert 'Height' column to float data type
test_data['Height'] = test_data['Height'].astype(float)
# Convert inches to meters
train_data_1['Height'] = train_data_1['Height'] * 0.0254
test_data['Height'] = test_data['Height'] * 0.0254

engineeering weight for train and test

In [21]:
# Remove 'lbs' from 'Weight' values and convert pounds to kilograms
train_data_1['Weight'] = train_data_1['Weight'].str.replace('lbs', '').astype(float) * 0.453592
# Convert 'Weight' column to float data type
train_data_1['Weight'] = train_data_1['Weight'].astype(float)
train_data_1['Weight'].isna().sum()
test_data['Weight'] = test_data['Weight'].str.replace('kg', '').astype(float)
test_data['Weight'] = test_data['Weight'].astype(float)

calculating BMI

In [22]:
train_data_1['BMI'] = train_data_1['Weight'] / (train_data_1['Height'] ** 2)
test_data['BMI'] = test_data['Weight'] / (test_data['Height'] ** 2)

calculating Potential difference

In [23]:
train_data_1['Potential Difference'] = train_data_1['Overall'] - train_data_1['Potential']
test_data['Potential Difference'] = test_data['Overall'] - test_data['Potential']

creating features based on the skills 

imputing and calculating average in the skills for train and test data

In [24]:
# Calculate average scores for attack, defense, midfield, and goalkeeping
train_data_1['Attack'] = train_data_1[['Finishing', 'HeadingAccuracy', 'ShortPassing', 'Volleys',
                                       'Dribbling', 'Curve', 'FKAccuracy', 'LongPassing', 'BallControl']].mean(axis=1)

train_data_1['Defense'] = train_data_1[['Marking', 'StandingTackle', 'SlidingTackle', 'Interceptions']].mean(axis=1)

train_data_1['Midfield'] = train_data_1[['Acceleration', 'SprintSpeed', 'Agility', 'Reactions', 'Balance',
                                         'ShotPower', 'Jumping', 'Stamina', 'Strength']].mean(axis=1)

train_data_1['Goalkeeping'] = train_data_1[['GKDiving', 'GKHandling', 'GKKicking', 'GKPositioning', 'GKReflexes']].mean(axis=1)

# Impute missing values in 'Crossing' based on 'Attack' features
crossing_features = ['Finishing', 'HeadingAccuracy', 'ShortPassing', 'Volleys', 'Dribbling', 'Curve', 'FKAccuracy',
                     'LongPassing', 'BallControl']
train_data_1['Crossing'] = train_data_1.apply(lambda row: row['Crossing'] if pd.notnull(row['Crossing'])
                                              else row[crossing_features].mean(), axis=1)

# Impute missing values in 'Defense' based on 'Defense' features
defense_features = ['Marking', 'StandingTackle', 'SlidingTackle', 'Interceptions']
train_data_1['Defense'] = train_data_1.apply(lambda row: row['Defense'] if pd.notnull(row['Defense'])
                                             else row[defense_features].mean(), axis=1)

# Impute missing values in 'Midfield' based on 'Midfield' features
midfield_features = ['Acceleration', 'SprintSpeed', 'Agility', 'Reactions', 'Balance', 'ShotPower',
                     'Jumping', 'Stamina', 'Strength']
train_data_1['Midfield'] = train_data_1.apply(lambda row: row['Midfield'] if pd.notnull(row['Midfield'])
                                              else row[midfield_features].mean(), axis=1)

# Impute missing values in 'Goalkeeping' based on 'Goalkeeping' features
goalkeeping_features = ['GKDiving', 'GKHandling', 'GKKicking', 'GKPositioning', 'GKReflexes']
train_data_1['Goalkeeping'] = train_data_1.apply(lambda row: row['Goalkeeping'] if pd.notnull(row['Goalkeeping'])
                                                 else row[goalkeeping_features].mean(), axis=1)
#doing the same with test_data
# Calculate average scores for attack, defense, midfield, and goalkeeping
test_data['Attack'] = test_data[['Finishing', 'HeadingAccuracy', 'ShortPassing', 'Volleys',
                                 'Dribbling', 'Curve', 'FKAccuracy', 'LongPassing', 'BallControl']].mean(axis=1)

test_data['Defense'] = test_data[['Marking', 'StandingTackle', 'SlidingTackle', 'Interceptions']].mean(axis=1)

test_data['Midfield'] = test_data[['Acceleration', 'SprintSpeed', 'Agility', 'Reactions', 'Balance',
                                   'ShotPower', 'Jumping', 'Stamina', 'Strength']].mean(axis=1)

test_data['Goalkeeping'] = test_data[['GKDiving', 'GKHandling', 'GKKicking', 'GKPositioning', 'GKReflexes']].mean(axis=1)

# Impute missing values in 'Crossing' based on 'Attack' features
test_crossing_features = ['Finishing', 'HeadingAccuracy', 'ShortPassing', 'Volleys', 'Dribbling', 'Curve', 'FKAccuracy',
                          'LongPassing', 'BallControl']
test_data['Crossing'] = test_data.apply(lambda row: row['Crossing'] if pd.notnull(row['Crossing'])
                                        else row[test_crossing_features].mean(), axis=1)

# Impute missing values in 'Defense' based on 'Defense' features
test_defense_features = ['Marking', 'StandingTackle', 'SlidingTackle', 'Interceptions']
test_data['Defense'] = test_data.apply(lambda row: row['Defense'] if pd.notnull(row['Defense'])
                                       else row[test_defense_features].mean(), axis=1)

# Impute missing values in 'Midfield' based on 'Midfield' features
test_midfield_features = ['Acceleration', 'SprintSpeed', 'Agility', 'Reactions', 'Balance', 'ShotPower',
                          'Jumping', 'Stamina', 'Strength']
test_data['Midfield'] = test_data.apply(lambda row: row['Midfield'] if pd.notnull(row['Midfield'])
                                        else row[test_midfield_features].mean(), axis=1)

# Impute missing values in 'Goalkeeping' based on 'Goalkeeping' features
test_goalkeeping_features = ['GKDiving', 'GKHandling', 'GKKicking', 'GKPositioning', 'GKReflexes']
test_data['Goalkeeping'] = test_data.apply(lambda row: row['Goalkeeping'] if pd.notnull(row['Goalkeeping'])
                                           else row[test_goalkeeping_features].mean(), axis=1)


doing some more eda/feature generation

In [25]:
train_data_1.columns

Index(['Unnamed: 0', 'ID', 'Name', 'Age', 'Photo', 'Nationality', 'Flag',
       'Overall', 'Potential', 'Club', 'Club Logo', 'Value', 'Wage', 'Special',
       'Preferred Foot', 'International Reputation', 'Weak Foot',
       'Skill Moves', 'Work Rate', 'Body Type', 'Real Face', 'Position',
       'Jersey Number', 'Joined', 'Loaned From', 'Contract Valid Until',
       'Height', 'Weight', 'Crossing', 'Finishing', 'HeadingAccuracy',
       'ShortPassing', 'Volleys', 'Dribbling', 'Curve', 'FKAccuracy',
       'LongPassing', 'BallControl', 'Acceleration', 'SprintSpeed', 'Agility',
       'Reactions', 'Balance', 'ShotPower', 'Jumping', 'Stamina', 'Strength',
       'LongShots', 'Aggression', 'Interceptions', 'Positioning', 'Vision',
       'Penalties', 'Composure', 'Marking', 'StandingTackle', 'SlidingTackle',
       'GKDiving', 'GKHandling', 'GKKicking', 'GKPositioning', 'GKReflexes',
       'Best Position', 'Best Overall Rating', 'year', 'Release Clause',
       'DefensiveAwareness', 'B

In [26]:
train_1=train_data_1[['Potential','Overall','Wage','International Reputation','Age','BMI','Potential Difference','Goalkeeping','Defense','Midfield','Attack']]
test_1=test_data[['Potential','Overall','Wage','International Reputation','Age','BMI','Potential Difference','Goalkeeping','Defense','Midfield','Attack']]

find which data types are object

In [27]:
train_1.head()

,Potential,Overall,Wage,International Reputation,Age,BMI,Potential Difference,Goalkeeping,Defense,Midfield,Attack
0,69,69,€7K,2.0,33,24.547741,0,11.2,69.250000,69.555556,64.000000
1,71,71,€7K,1.0,37,20.518136,0,11.2,56.000000,62.000000,70.888889
2,71,71,€15K,2.0,33,23.775280,0,10.2,34.500000,56.666667,71.555556
3,70,70,€4K,1.0,35,22.377775,0,69.2,16.000000,50.111111,16.777778
4,82,68,€0,1.0,36,23.012574,-14,26.4,48.333333,59.333333,66.428571


train feature

In [28]:
y_train=train_data_1['Value']

converting Value and Wage as per

In [29]:
import re
# Define a function to preprocess the 'Wage' column
def preprocess_wage(value):
    value = re.sub(r'€', '', value)  # Remove euro symbol
    if 'K' in value:
        value = float(re.sub(r'[^0-9.]', '', value)) * 1000  # Remove 'K' and multiply by 1000
    elif 'M' in value:
        value = float(re.sub(r'[^0-9.]', '', value)) * 1000000  # Remove 'M' and multiply by 1000000
    else:
        value = float(value)
    return value

# Apply the preprocessing function to 'Wage' column in train_1 dataset
train_1['Wage'] = train_1['Wage'].apply(preprocess_wage)

# Convert the column to float data type
train_1['Wage'] = train_1['Wage'].astype(float)

# Apply the preprocessing function to 'Wage' column in train_1 dataset
test_1['Wage'] = test_1['Wage'].apply(preprocess_wage)

# Convert the column to float data type
test_1['Wage'] = test_1['Wage'].astype(float)

#Applying it to y_train too
y_train=y_train.apply(preprocess_wage)

#Convert the column to float data type
y_train=y_train.astype(float)

/tmp/ipykernel_20/3838849759.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_1['Wage'] = train_1['Wage'].apply(preprocess_wage)
/tmp/ipykernel_20/3838849759.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_1['Wage'] = train_1['Wage'].astype(float)
/tmp/ipykernel_20/3838849759.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydat

fitting model with  best parameters

In [30]:
from xgboost import XGBRegressor
model = XGBRegressor(learning_rate=0.05, n_estimators=500)
model.fit(train_1, y_train)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.05, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             n_estimators=500, n_jobs=None, num_parallel_tree=None,
             predictor=None, random_state=None, ...)

In [31]:
pred=model.predict(test_1)

In [32]:
output = pd.DataFrame({'Photo':test_data['Photo'],'Value': pred})  # Create the DataFrame with the 'Value' column
output.head()

,Photo,Value
0,https://cdn.sofifa.com/players/212/198/22_60.png,71609544.0
1,https://cdn.sofifa.com/players/209/658/22_60.png,54067988.0
2,https://cdn.sofifa.com/players/176/580/22_60.png,30014316.0
3,https://cdn.sofifa.com/players/192/985/22_60.png,84322752.0
4,https://cdn.sofifa.com/players/224/334/22_60.png,27655076.0


hence output submission has been created

converting to a csv as needed

In [33]:
output.to_csv('submissions.csv',index=False)
print("Submission saved successfully")

Submission saved successfully
